# Installation and Setup

In [ ]:
# Import required packages
import os
import torch
import torchrec
import fbgemm_gpu
from utils.environment import TorchRecEnvironment

## Environment Check

In [ ]:
env_checker = TorchRecEnvironment()
env_status = env_checker.check_environment()

print("Environment Status:")
print(f"CUDA Available: {env_status['cuda_available']}")
print(f"GPU Count: {env_status['gpu_count']}")
print(f"Distributed Available: {env_status['distributed_available']}")
print(f"FBGEMM Available: {env_status['fbgemm_available']}")

## Basic TorchRec Test

In [ ]:
from torchrec import EmbeddingBagCollection
from torchrec.modules.embedding_configs import EmbeddingBagConfig
from torchrec.sparse.jagged_tensor import KeyedJaggedTensor

# Create a simple embedding bag configuration
eb_configs = [
    EmbeddingBagConfig(
        name="simple_embedding",
        embedding_dim=16,
        num_embeddings=100,
        feature_names=["feature1"]
    )
]

# Initialize on CPU first
ebc = EmbeddingBagCollection(
    tables=eb_configs,
    device=torch.device("cpu")
)

## Create Test Input

In [ ]:
# Create test input
values = torch.tensor([1, 2, 3, 4, 5])
lengths = torch.tensor([2, 3])  # First sequence has 2 elements, second has 3

# Create KeyedJaggedTensor
kjt = KeyedJaggedTensor.from_lengths_sync(
    keys=["feature1"],
    values=values,
    lengths=lengths
)

# [MARKDOWN: Test Forward Pass]

# Perform forward pass
output = ebc(kjt)

# Verify output
print("\nOutput Verification:")
print(f"Output type: {type(output)}")
print(f"Output keys: {output.keys()}")
print(f"Output values shape: {output.values().shape}")

## GPU Test (if available)

In [ ]:
if torch.cuda.is_available():
    # Move model to GPU
    ebc_gpu = EmbeddingBagCollection(
        tables=eb_configs,
        device=torch.device("cuda")
    )
    
    # Move input to GPU
    kjt_gpu = KeyedJaggedTensor.from_lengths_sync(
        keys=["feature1"],
        values=values.cuda(),
        lengths=lengths
    )
    
    # Test forward pass on GPU
    output_gpu = ebc_gpu(kjt_gpu)
    
    print("\nGPU Output Verification:")
    print(f"Output device: {output_gpu.values().device}")
    print(f"Output shape: {output_gpu.values().shape}")

## Memory Test

In [ ]:
if torch.cuda.is_available():
    from utils.debugging import TorchRecDebugger
    
    debugger = TorchRecDebugger()
    
    print("\nMemory Status:")
    memory_stats = debugger.memory_status()
    print(f"Allocated: {memory_stats['allocated'] / 1e6:.2f} MB")
    print(f"Reserved: {memory_stats['reserved'] / 1e6:.2f} MB")
    print(f"Max Allocated: {memory_stats['max_allocated'] / 1e6:.2f} MB")

## Cleanup and Reset

In [ ]:
# Clear memory
if torch.cuda.is_available():
    debugger.clear_memory()

## Complete Installation Checklist

In [ ]:
checklist = {
    "pytorch_installed": True,
    "torchrec_installed": True,
    "fbgemm_installed": True,
    "cuda_available": torch.cuda.is_available(),
    "can_create_ebc": True,
    "can_forward_pass": True,
    "gpu_working": torch.cuda.is_available() and torch.cuda.device_count() > 0
}

print("\nInstallation Checklist:")
for check, status in checklist.items():
    print(f"{check}: {'✅' if status else '❌'}")